In [96]:
# Importar librerías necesarias

import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [28]:
# Importar datos

df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_592/607337228.py:3: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


# Consignas

Columna “reservation_status_date” (CARLA)

1. Limpieza y transformación: 
    -  Esta columna es de tipo object y deberíamos pasarla a tipo date. Para poder hacerlo, vamos a necesitar unificar el formato en el que aparecen las fechas (en algunos casos tienen guiones y en otros barras).
    - Hay 30 filas con fechas posteriores a 2017. En principio no nos preocupa mucho por tratarse de un porcentaje muy pequeño. Intentaremos corregirlo teniendo en cuenta las columnas que tienen información de la fecha de arrival. 

2. Imputación de nulos: 
    - Completar a partir de “arrival _date_year”
    - El resto imputarlas con KNN o iterative.


## Exploración y análisis

In [15]:
# Definir una función para explorar las caraterísticas de una columna rápidamente

def quick_check(df, column_name):
        """ This function is for testing purposes, to quicky check data type and unique values of a column"""
        print (f"\nColumn name: {column_name}")
        print (f"\nData type: {df[column_name].dtype}")
        print (f"\nNot null count: {df[column_name].notnull().sum()}")
        print (f"\nNull count: {df[column_name].isnull().sum()}, {round(df[column_name].isnull().sum()/df.shape[0]*100)}%")
        print (f"\nDuplicated values: {df[column_name].duplicated().sum()}")
        print (f"\nUnique values: \n{df[column_name].unique()}")
        print (f"\nValue counts: \n{df[column_name].value_counts()}")      

In [16]:
# Explorar columna "reservation_status_date"

quick_check(df,"reservation_status_date")


Column name: reservation_status_date

Data type: object

Not null count: 106514

Null count: 76363, 42%

Duplicated values: 181885

Unique values: 
['2015-07-01 00:00:00' '2015-07-02 00:00:00' '2015-07-03 00:00:00'
 '2015-05-06 00:00:00' '2015-04-22 00:00:00' '2015-06-23 00:00:00'
 '2015-07-05 00:00:00' nan '2015-07-07 00:00:00' '2015-07-08 00:00:00'
 '2015-05-11 00:00:00' '2015-07-15 00:00:00' '2015-07-16 00:00:00'
 '2015-05-29 00:00:00' '2015-05-19 00:00:00' '2015-06-19 00:00:00'
 '2015-07-06 00:00:00' '2015-05-23 00:00:00' '2015-05-18 00:00:00'
 '2015-07-09 00:00:00' '2015-06-02 00:00:00' '2015-07-04 00:00:00'
 '2015-06-29 00:00:00' '2015-06-16 00:00:00' '2015-06-18 00:00:00'
 '2015-06-12 00:00:00' '2015-06-09 00:00:00' '2015-05-26 00:00:00'
 '2015-07-11 00:00:00' '2015-07-12 00:00:00' '2015-07-13 00:00:00'
 '2015-07-17 00:00:00' '2015-04-15 00:00:00' '2015-05-13 00:00:00'
 '2015-07-10 00:00:00' '2015-05-20 00:00:00' '2015-05-12 00:00:00'
 '2015-07-14 00:00:00' '2015-06-17 00:00:00

## Conclusiones iniciales (parciales): 
- Algunas fechas tienen formato fecha y hora 'AAAA-MM-DD 00:00:00' y otras solo fecha '2023-04-31'
- En el caso de las que tienen fecha y hora, la hora es siempre '00:00:00'

In [18]:
# Explorar las filas que tienen el formato 'AAAA-MM-DD' ya que parece haber errores en ellas 

# Filtrar dataframe por las filas que coinciden con el formato 'AAAA-MM-DD' en la columna 'reservation_status_date'  
df_filtered = df[df['reservation_status_date'].str.match(r'^\d{4}-\d{2}-\d{2}$', na=False)]

# Revisar columna 'reservation_status_date' del df filtrado
quick_check(df_filtered, 'reservation_status_date')



Column name: reservation_status_date

Data type: object

Not null count: 45

Null count: 0, 0%

Duplicated values: 0

Unique values: 
['2030-02-30' '2030-11-31' '2029-04-31' '2030-06-31' '2030-09-31'
 '2025-02-30' '2029-11-31' '2023-06-31' '2029-06-31' '2029-02-30'
 '2022-09-31' '2024-09-31' '2028-04-31' '2024-06-31' '2025-11-31'
 '2026-09-31' '2027-09-31' '2023-11-31' '2028-09-31' '2028-02-30'
 '2026-06-31' '2024-04-31' '2023-04-31' '2025-04-31' '2027-04-31'
 '2022-06-31' '2024-02-30' '2025-09-31' '2027-02-30' '2030-04-31'
 '2023-09-31' '2027-06-31' '2026-11-31' '2028-11-31' '2028-06-31'
 '2026-04-31' '2022-11-31' '2025-06-31' '2029-09-31' '2023-02-30'
 '2026-02-30' '2022-04-31' '2022-02-30' '2027-11-31' '2024-11-31']

Value counts: 
2030-02-30    1
2025-04-31    1
2022-06-31    1
2024-02-30    1
2025-09-31    1
2027-02-30    1
2030-04-31    1
2023-09-31    1
2027-06-31    1
2026-11-31    1
2028-11-31    1
2028-06-31    1
2026-04-31    1
2022-11-31    1
2025-06-31    1
2029-09-31    

In [20]:
# Inspeccionar df filtrado 

df_filtered.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
313,Resort Hotel,False,90.0,NaN,July,29.0,12.0,2.0,5.0,2.0,...,5.0,NaN,0.0,NaN,117.22,0.0,1.0,Check-Out,2030-02-30,NaN
7646,Resort Hotel,True,165.0,NaN,August,34.0,18.0,0.0,3.0,2.0,...,240.0,NaN,0.0,Transient,222.00,0.0,0.0,Canceled,2030-11-31,NaN
9139,Resort Hotel,True,16.0,NaN,October,NaN,29.0,0.0,1.0,2.0,...,NaN,NaN,0.0,Transient,38.00,0.0,1.0,Canceled,2029-04-31,NaN
9493,Resort Hotel,True,183.0,NaN,December,NaN,3.0,2.0,1.0,2.0,...,38.0,NaN,0.0,NaN,72.00,0.0,0.0,Canceled,2030-06-31,NaN
18350,Resort Hotel,True,82.0,2016.0,2,7.0,7.0,2.0,0.0,2.0,...,67.0,NaN,0.0,NaN,86.00,0.0,0.0,Canceled,2030-09-31,NaN


In [21]:
# Conocer la cantidad de filas que tiene el df filtrado

df_filtered.shape

(45, 32)

## Conclusiones finales

- Todas las fechas con formato 'AAAA-MM-DD' (45) son erróneas ya que contienen días que no existen en el mes, como 31/09 o 30/02 o son fechas de años en el futuro

## Decisión 

- Reemplazar las fechas erróneas, y también las nulas, por una categoría diferente para que no intervengan negativamente en el análisis


## Acciones de limpieza y transformación

1. Reemplazar las fechas erróneas con una categoría especial "1677-09-21" (es la fecha mínima que existe en Pandas) 
2. Reemplazar los nulos también con "1677-09-21"
3. Quitar la componente 'HH:MM:SS' (al ser todas 00:00:00 no aportan valor)
4. Cambiar el formato de la columna a "datetime" (usar to_datetime)


In [65]:
df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)

/tmp/ipykernel_592/1059111946.py:1: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/finanzas_hotel_bookings.csv", index_col=0)


In [66]:
# Definición de funciones de transformación

# Función para reemplazar las fechas erróneas con "0000-00-00"
def reemplazar_fechas_erroneas(fecha):
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        # Expresión regular para identificar fechas en formato 'AAAA-MM-DD'
        patron = r'^\d{4}-\d{2}-\d{2}$'
        if re.match(patron, fecha):
            return '1677-09-22'
    # Si no es una cadena de texto o no coincide con el patrón, devolver el valor original
    return fecha

# Función para transformar fechas con formato 'AAAA-MM-DD 00:00:00' en 'AAAA-MM-DD'
def quitar_horas (fecha):
    # Verificar si el valor es una cadena de texto
    if isinstance(fecha, str):
        fecha = fecha.split()[0]
        return fecha
    # Si no es una cadena de texto, devolver el valor original
    return fecha

In [67]:
# Aplicar las transformación a la columna

column = 'reservation_status_date'
df[column] = df[column].apply(reemplazar_fechas_erroneas)
df[column] = df[column].apply(quitar_horas)
df[column] = df[column].fillna('1677-09-22')
df[column] = pd.to_datetime(df[column])


In [84]:
# Print for testing 

quick_check(df, column)


Column name: reservation_status_date

Data type: datetime64[ns]

Not null count: 182877

Null count: 0, 0%

Duplicated values: 181930

Unique values: 
['2015-07-01T00:00:00.000000000' '2015-07-02T00:00:00.000000000'
 '2015-07-03T00:00:00.000000000' '2015-05-06T00:00:00.000000000'
 '2015-04-22T00:00:00.000000000' '2015-06-23T00:00:00.000000000'
 '2015-07-05T00:00:00.000000000' '1677-09-22T00:00:00.000000000'
 '2015-07-07T00:00:00.000000000' '2015-07-08T00:00:00.000000000'
 '2015-05-11T00:00:00.000000000' '2015-07-15T00:00:00.000000000'
 '2015-07-16T00:00:00.000000000' '2015-05-29T00:00:00.000000000'
 '2015-05-19T00:00:00.000000000' '2015-06-19T00:00:00.000000000'
 '2015-07-06T00:00:00.000000000' '2015-05-23T00:00:00.000000000'
 '2015-05-18T00:00:00.000000000' '2015-07-09T00:00:00.000000000'
 '2015-06-02T00:00:00.000000000' '2015-07-04T00:00:00.000000000'
 '2015-06-29T00:00:00.000000000' '2015-06-16T00:00:00.000000000'
 '2015-06-18T00:00:00.000000000' '2015-06-12T00:00:00.000000000'
 '2

## Imputación de nulos en la columna "reservation_status_date"

### Imputar 'reservation_status_date' a partir de la fecha de arrival

Para poder hacer esta imputación, es necesario conocer el 'reservation_status' asociado:

- Si el reservation status es "Check-out", entonces la "reservation_status_date" es igual a "arrival_date" + "total_stays".
- Si el reservation status es "No-Show", entonces la "reservation_status_date" a "arrival_date".
- Si el reservation status es "Canceled", no es posible deducir la "reservation_status_date".

In [91]:
# ¿Cuántos registros con reservation status "Check-out" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'Check-out') & (df['reservation_status_date'] == '1677-09-22T00:00:00.000000000')].shape[0]


0

In [92]:
# ¿Cuántos registros con reservation status "No-show" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'No-show') & (df['reservation_status_date'] == '1677-09-22T00:00:00.000000000')].shape[0]

0

In [94]:
# ¿Cuántos registros con reservation status "Canceled" tienen "reservation_status_date" nulo?

df[(df['reservation_status'] == 'Canceled') & (df['reservation_status_date'] == '1677-09-22T00:00:00.000000000')].shape[0]

4682

### CONCLUSIÓN: No es posible imputar la 'reservation_status_date' a partir de la fecha de arrival ❌❌❌

Esto se debe a que los registros que tienen 'reservation_status_date' tienen el 'reservation_status' "Canceled" o Null.

## Acciones de imputación de nulos

- Imputar con KNN o iterative.